<a href="https://colab.research.google.com/github/MilesCrossen/STEP-Bayesian-Model/blob/main/STEPBayesianProcessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np # Cool math features
from scipy.stats import norm # Stats, normal distribution
import pandas as pd # For manipulating CSVs

modes = { # Transport modes. Format: mean, standard deviation
    "Walking": {"mean": 5, "std": 1},
    "Cycling": {"mean": 15, "std": 4},
    "Bus": {"mean": 20, "std": 6},
    "Driving": {"mean": 50, "std": 15},
    "Train": {"mean": 100, "std": 20}
}

prior_probs = {mode: 1/ len(modes) for mode in modes} # At first priors are just equal to one
#another

EPSILON = 1e-8 # Had this issue before where we'd have zero probabilities so therefore
#could reach a point of no return (i.e. where walking equals 0.0000...%) so this is a floor

def bayesian_update(observed_speed, prior_probs): # This is how we do function stuff in python
    """Performs one Bayesian update given observed speed and prior probabilities."""
    likelihoods = {}
    posteriors = {}

    for mode, stats in modes.items():#  Calc likelihoods w/epsilon floor
        mu = stats["mean"]
        sigma = stats["std"]
        raw_likelihood = norm.pdf(observed_speed, loc=mu, scale=sigma)
        likelihoods[mode] = max(raw_likelihood, EPSILON)

    # This is for finding denominator of Bayes'
    evidence = sum(max(prior_probs[mode], EPSILON) * likelihoods[mode] for mode in modes)

    for mode in modes:
        prior = max(prior_probs[mode], EPSILON)
        posterior = (likelihoods[mode] * prior) / evidence
        posteriors[mode] = posterior

    df = pd.DataFrame({ #For displayying results
        "Mode": list(modes.keys()),
        "Prior": [prior_probs[mode] for mode in modes],
        "Likelihood": [likelihoods[mode] for mode in modes],
        "Posterior": [posteriors[mode] for mode in modes]
    })

    print("\nBayesian Update Result for Speed = {:.2f} km/h:".format(observed_speed))
    print(df.to_string(index=False, float_format="{:.10f}".format))
    return posteriors# This becomes the next prior


if __name__ == "__main__": # Repeating loop thingy
    current_priors = prior_probs
    try:
        while True:
            user_input = input("\nEnter observed speed in km/h: ")
            try:
                speed = float(user_input)
                current_priors = bayesian_update(speed, current_priors)
            except ValueError:
                print("Invalid input. Please enter a numeric speed.")
    except KeyboardInterrupt:
        print("\nTerminated by user.")


Terminated by user.
